In [64]:
import pandas as pd
import os
import numpy as np

In [65]:
# File Paths
absolute_path = "C:\\Users\\tykun\\\OneDrive\\Documents\\SchoolDocs\VSCodeProjects\\connectedData\\board_analysis\\"
altered_dataframes = "altered_dataframes\\"
gpt_dataframes = "gpt_dataframes\\"
graphs = "graphs\\"
scripts =  "scripts\\"
board_dataframes = "board_dataframes\\"
temporary = "temporary_data\\"
final_scripts = "final_scripts\\"
normalized_dataframes = "normalized_dataframes\\"
college_matching = "college_matching\\"
regression = "regression\\"
normalized_dataframes = "normalized_dataframes\\"
normalized_regression_boards = "normalized_regression_boards\\"
network = "network\\"
network_boards = "network_boards\\"

altered_dataframe_path = f"{absolute_path}{altered_dataframes}"
gpt_dataframe_path = f"{absolute_path}{gpt_dataframes}" 
graph_path = f"{absolute_path}{graphs}"
script_path = f"{absolute_path}{scripts}"
boards_path = f"{absolute_path}{board_dataframes}"
temporary_data_path = f"{absolute_path}{temporary}"
regression_path =  f"{absolute_path}{final_scripts}{regression}"
regression_boards_path = f"{regression_path}{normalized_regression_boards}"

In [66]:
#2003-2019
shanghai_rankings_path = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\board_analysis\\final_scripts\\rankings\\ShanghaiUniversityRanking\\"
#2011-2020
times_rankings_path = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\board_analysis\\final_scripts\\rankings\\TimesUniversityRankings\\"

In [67]:
times_file_path = os.path.join(times_rankings_path, "TimesWorldRanking2011.hdf")
times_rankings = pd.read_hdf(times_file_path)
# print(times_rankings.head)

In [68]:
shanghai_file_path = os.path.join(shanghai_rankings_path, "ShanghaiUniversityRanking_ShanghaiRanking2004.hdf")
shanghai_rankings = pd.read_hdf(shanghai_file_path)

print(shanghai_rankings.columns)

Index(['Rank', 'FullName', 'href', 'Country', 'TotalScore', 'Alumni', 'Award',
       'HiCi', 'N&S', 'Pub', 'PCP', 'AffiliationID'],
      dtype='object')


In [69]:

university_stats_path = os.path.join(regression_path, "regression_stats", "regression_university_board_statistics.csv")
df = pd.read_csv(university_stats_path)
df["Year"] = df["Year"].astype(str)

if "Rank" in df.columns:
    df = df.drop("Rank", axis=1)

valid_years = set(str(year) for year in range(2003, 2020))
df["ShanghaiYear"] = df["Year"].apply(lambda y: y if y in valid_years else "2003")

merged_list = []
for sh_year in sorted(df["ShanghaiYear"].unique()):
    shanghai_file_path = os.path.join(shanghai_rankings_path, f"ShanghaiUniversityRanking_ShanghaiRanking{sh_year}.hdf")
    shanghai_rankings = pd.read_hdf(shanghai_file_path)
    shanghai_rankings.rename(columns={"AffiliationID": "AffiliationId"}, inplace=True)
    shanghai_subset = shanghai_rankings[["AffiliationId", "Rank"]].copy()
    shanghai_subset["ShanghaiYear"] = sh_year  

    subset = df[df["ShanghaiYear"] == sh_year].copy()
    merged = pd.merge(subset, shanghai_subset, on=["AffiliationId", "ShanghaiYear"], how="left")
    merged_list.append(merged)

merged_df = pd.concat(merged_list, ignore_index=True)
mapping_2003 = merged_df.loc[merged_df["Year"] == "2003"].set_index("AffiliationId")["Rank"]

def fill_rank(row):
    if pd.isna(row["Rank"]) and int(row["Year"]) < 2003:
        if row["AffiliationId"] in mapping_2003.index:
            return mapping_2003.loc[row["AffiliationId"]]
    return row["Rank"]

merged_df["Rank"] = merged_df.apply(fill_rank, axis=1)
merged_df.drop("ShanghaiYear", axis=1, inplace=True)
university_board_statistics_df = merged_df.copy()

def clean_rank(x):
    if pd.isna(x):
        return np.nan
    x_str = str(x)
    if '-' in x_str:
        x_str = x_str.split('-')[0]
    try:
        return float(x_str)
    except:
        return np.nan
    
university_board_statistics_df["Rank"] = university_board_statistics_df["Rank"].apply(clean_rank)
university_board_statistics_df["Rank"] = university_board_statistics_df["Rank"].fillna(500)
university_board_statistics_df["Rank"] = university_board_statistics_df["Rank"].astype(int)
print(university_board_statistics_df.columns)


Index(['Year', 'Institution', 'AffiliationId', 'female_president',
       'PrimarySample', 'total_members', 'total_ethnicity', 'board_turnover',
       'carnegie_id', 'state', 'control', 'StateSystem', 'region',
       'num_billionaires', 'student.women', 'faculty.women',
       'faculty.race_ethnicity.white', 'student.size',
       'cost.tuition.out_of_state', 'school.faculty_salary', 'RD_expenditure',
       'female_proportion', 'poc_proportion', 'billionaire_proportion',
       'eigenvector', 'betweenness', 'degree', 'strength', 'clustering',
       'Rank'],
      dtype='object')


In [70]:
def classify_rank(x):
    if pd.isna(x):
        return 101
    x_str = str(x)
    if '-' in x_str:
        x_str = x_str.split('-')[0]
    try:
        rank_val = float(x_str)
    except ValueError:
        return '101'
    if rank_val <= 10:
        return '1 to 10'
    elif rank_val <= 30:
        return '11 to 30'
    elif rank_val <= 80:
        return '31 to 80'
    elif rank_val <= 100:
        return '81 to 100'
    else:
        return '101'

university_board_statistics_df["Rank"] = university_board_statistics_df["Rank"].apply(classify_rank)
university_board_statistics_df.drop_duplicates(inplace=True)
university_board_statistics_df.to_csv(university_stats_path, index=False)
